In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cluster import MiniBatchKMeans
import warnings
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns

In [17]:
train = pd.read_csv("train.csv")

In [18]:
test = pd.read_csv("data_for_test.csv")

In [4]:
submit = pd.read_csv("sample_submission.csv")

In [119]:
# def preprocessing(df_train, df_test):
#     """
#     normalizing continuous
#     adding features
#     normalizing continuous variables
#     """
#     df_train[['pid','reading_count']] = df_train['key'].str.split('-', expand=True)
#     df_train = df_train.drop('pid', axis=1)
#     df_train['reading_count'] = df_train['reading_count'].astype(int)
    
#     df_test[['pid','reading_count']] = df_test['key'].str.split('-', expand=True)
#     df_test = df_test.drop('pid', axis=1)
#     df_test['reading_count'] = df_test['reading_count'].astype(int)
       
#     #normalize
#     cols = ['xx1','xx2','xx3','xx4','xx5'] 
#     for col in cols:
#         df_test[col] = (df_test[col] - df_train[col].mean())/df_train[col].std()
#         df_train[col] = (df_train[col] - df_train[col].mean())/df_train[col].std()
#     return df_train, df_test

In [21]:
train = train.groupby('key').mean().reset_index()

In [24]:
test = test.groupby('key').mean().reset_index()

In [22]:
train.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,y_mean_MAP,y_mean_HR
0,0-28,0,0,72,39,5,1,0,0,0,80.166667,99.733333,113.433333,69.566667,87.000000,86.426667,79.130000
1,1-10,1,1,64,55,9,5,1,0,1,72.100000,99.800000,113.900000,59.600000,76.466667,73.033333,72.920000
2,1-11,1,1,64,55,9,5,1,0,1,71.500000,99.966667,114.066667,59.500000,75.400000,77.560000,72.493333
3,1-14,1,1,64,55,9,5,1,0,1,99.133333,100.000000,133.433333,69.833333,84.966667,82.523333,118.613333
4,1-15,1,1,64,55,9,5,1,0,1,150.200000,100.000000,123.733333,77.100000,89.233333,98.706667,189.720000


In [27]:
test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5
0,1005-1,1005,0,80,41,4,2,1,0,0,57.866667,98.433333,126.566667,42.800000,71.466667
1,1005-10,1005,0,80,41,4,2,0,0,0,75.766667,97.333333,126.933333,48.400000,71.866667
2,1005-11,1005,0,80,41,4,2,1,0,0,65.666667,92.166667,109.000000,48.633333,65.600000
3,1005-12,1005,0,80,41,4,2,1,0,0,64.600000,94.933333,124.200000,55.400000,75.133333
4,1005-13,1005,0,80,41,4,2,0,0,0,73.800000,97.400000,134.733333,52.366667,77.200000


In [28]:
train_MAP = train.drop(['y_mean_HR','key'], axis = 1)
train_HR = train.drop(['y_mean_MAP','key'], axis = 1)

In [29]:
valid_idx = range(int(train.shape[0]*80/100), train.shape[0])
train_idx = range(int(train.shape[0]*80/100))

In [30]:
valid_MAP = train_MAP.loc[valid_idx,:]
train_MAP = train_MAP.loc[train_idx,:]
valid_HR = train_HR.loc[valid_idx,:]
train_HR = train_HR.loc[train_idx,:]

## xgb on MAP

In [31]:
model = GradientBoostingRegressor(learning_rate=0.1, max_depth=4, n_estimators=200, min_samples_leaf=5)
model.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.802949541361218

In [57]:
model1 = GradientBoostingRegressor(learning_rate=0.1, max_depth=1, n_estimators=500, min_samples_leaf=5, loss='ls')
model1.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model1.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.8043587148257328

In [58]:
model2 = GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=750)
model2.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model2.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.7982998985173393

## rf on MAP

In [41]:
from sklearn.ensemble import RandomForestRegressor

In [42]:
model_rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=4,
                                 max_features='sqrt', random_state=42, verbose=True)
model_rf.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model_rf.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


0.7984063586950956

In [43]:
model_rf_1 = RandomForestRegressor(n_estimators=500, min_samples_leaf=3,
                                 max_features='sqrt', random_state=42, verbose=True, n_jobs=-1)
model_rf_1.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model_rf_1.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.2s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.1s finished


0.7995143414323127

### BEST model for MAP is xgb model

In [44]:
MAP_predictions = model.predict(test.drop('key', axis=1))

## xgb on HR

In [45]:
model_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=4, n_estimators=200, min_samples_leaf=5)
model_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.9209887378341927

In [59]:
model1_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=1, n_estimators=500, min_samples_leaf=3)
model1_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model1_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.9206379481633227

In [47]:
model2_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=1, n_estimators=1000)
model2_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model2_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.9206864481953145

## rf on HR

In [63]:
model_HR_rf = RandomForestRegressor(n_estimators=1000, min_samples_leaf=10,
                                    max_features='auto', random_state=42, verbose=True, n_jobs=-1)
model_rf_1.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model_rf_1.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.1s finished


0.8963462010521809

### Best model for HR is xbg model

In [50]:
HR_predictions = model_HR.predict(test.drop('key', axis=1))

In [51]:
test['y_mean_MAP'] = MAP_predictions
test['y_mean_HR'] = HR_predictions

In [52]:
toSubmit = test[['key','y_mean_MAP','y_mean_HR']]

In [53]:
toSubmit = toSubmit.groupby('key').mean()

In [54]:
toSubmit.to_csv("no_one_2.csv")